In [1]:
import pandas as pd
import numpy as np
import re

pd.options.plotting.backend = 'plotly'

### Example: Global warming 🌎

Consider the following three documents.

In [2]:
sentences = pd.Series([
    'I really really want global peace',
    'I must enjoy global warming',
    'We must solve climate change'
])

sentences

0    I really really want global peace
1          I must enjoy global warming
2         We must solve climate change
dtype: object

Let's represent each document using the bag of words model.

In [3]:
unique_words = pd.Series(sentences.str.split().sum()).value_counts()
unique_words

really     2
must       2
I          2
global     2
want       1
climate    1
solve      1
warming    1
peace      1
change     1
enjoy      1
We         1
dtype: int64

In [4]:
counts_dict = {}
for word in unique_words.index:
    re_pat = fr'\b{word}\b'
    counts_dict[word] = sentences.str.count(re_pat).astype(int).tolist()
    
counts_df = pd.DataFrame(counts_dict).set_index(sentences)

In [5]:
counts_df

,really,must,I,global,want,climate,solve,warming,peace,change,enjoy,We
I really really want global peace,2,0,1,1,1,0,0,0,1,0,0,0
I must enjoy global warming,0,1,1,1,0,0,0,1,0,0,1,0
We must solve climate change,0,1,0,0,0,1,1,0,0,1,0,1


Let's now find the cosine similarity between each document.

In [6]:
counts_df

,really,must,I,global,want,climate,solve,warming,peace,change,enjoy,We
I really really want global peace,2,0,1,1,1,0,0,0,1,0,0,0
I must enjoy global warming,0,1,1,1,0,0,0,1,0,0,1,0
We must solve climate change,0,1,0,0,0,1,1,0,0,1,0,1


In [7]:
def sim_pair(s1, s2):
    return np.dot(s1, s2) / (np.linalg.norm(s1) * np.linalg.norm(s2))

In [8]:
# Look at the documentation of the .corr method to see how this works!
counts_df.T.corr(sim_pair)

,I really really want global peace,I must enjoy global warming,We must solve climate change
I really really want global peace,1.000000,0.316228,0.0
I must enjoy global warming,0.316228,1.000000,0.2
We must solve climate change,0.000000,0.200000,1.0


**Issue:** Bag of words only encodes the **words** that each document uses, not their **meanings**.
- "I really really want global peace" and "We must solve climate change" have similar meanings, but have no shared words, and thus a low cosine similarity.
- "I really really want global peace" and "I must enjoy global warming" have very different meanings, but a relatively high cosine similarity.

### Pitfalls of the bag of words model

Remember, the key assumption underlying the bag of words model is that **two documents are similar if they share many words in common**.

- The bag of words model doesn't consider **order**.
    - The job titles `'deputy fire chief'` and `'chief fire deputy'` are treated as the same.

- The bag of words model doesn't consider the **meaning** of words.
    - `'I love data science'` and `'I hate data science'` share 75% of their words, but have very different meanings.

- The bag of words model treats all words as being equally important.
    - `'deputy'` and `'fire'` have the same importance, even though `'fire'` is probably more important in describing someone's job title.
    - <span style="color:red"><b>Let's address this point.</b></span>

## TF-IDF

### The importance of words

**Issue:** The bag of words model doesn't know which words are "important" in a document. Consider the following document:

<center>"my brother has a friend named <b>sam</b> who has an uncle named <b>sam</b>"</center>

How do we determine which words are important?
- The most common words ("the", "has") often **don't** have much meaning!
- The very rare words are also less important!

**Goal:** Find a way of quantifying the importance of a word in a document by **balancing the above two factors**, i.e. find the word that **best summarizes** a document.

### Term frequency

- The **term frequency** of a word (term) $t$ in a document $d$, denoted $\text{tf}(t, d)$ is the proportion of words in document $d$ that are equal to $t$.

$$\text{tf}(t, d) = \frac{\text{# of occurrences of $t$ in $d$}}{\text{total # of words in $d$}}$$

- Intuition: Words that occur often within a document are important to the document's meaning.
    - If $\text{tf}(t, d)$ is large, then word $t$ occurs often in $d$.
    - If $\text{tf}(t, d)$ is small, then word $t$ does not occur often $d$.

### Inverse document frequency

- The **inverse document frequency** of a word $t$ in a set of documents $d_1, d_2, ...$ is

$$\text{idf}(t) = \log \left(\frac{\text{total # of documents}}{\text{# of documents in which $t$ appears}} \right)$$

- Intuition: If a word appears in every document (like "the" or "has"), it is probably not a good summary of any one document.
    - If $\text{idf}(t)$ is large, then $t$ is rarely found in documents.
    - If $\text{idf}(t)$ is small, then $t$ is commonly found in documents.
    - Think of $\text{idf}(t)$ as the "rarity factor" of $t$ across documents – the larger $\text{idf}(t)$ is, the more rare $t$ is.

### Intuition

$$\text{tf}(t, d) = \frac{\text{# of occurrences of $t$ in $d$}}{\text{total # of words in $d$}}$$

$$\text{idf}(t) = \log \left(\frac{\text{total # of documents}}{\text{# of documents in which $t$ appears}} \right)$$

**Goal:** Quantify how well word $t$ **summarizes** document $d$.

### Term frequency-inverse document frequency

The **term frequency-inverse document frequency (TF-IDF)** of word $t$ in document $d$ is the product:

$$
\begin{align*}\text{tfidf}(t, d) &= \text{tf}(t, d) \cdot \text{idf}(t) \\\ &= \frac{\text{# of occurrences of $t$ in $d$}}{\text{total # of words in $d$}} \cdot \log \left(\frac{\text{total # of documents}}{\text{# of documents in which $t$ appears}} \right) \end{align*} $$

- If $\text{tfidf}(t, d)$ is large, then $t$ is a good summary of $d$, because $t$ occurs often in $d$ but rarely across all documents.

- TF-IDF is a **heuristic** – it has no probabilistic justification.

- To know if $\text{tfidf}(t, d)$ is large for one particular word $t$, we need to compare it to $\text{tfidf}(t_i, d)$, for several different words $t_i$.

### Computing TF-IDF

**Question:** What is the TF-IDF of "global" in the second sentence?

In [9]:
sentences

0    I really really want global peace
1          I must enjoy global warming
2         We must solve climate change
dtype: object

**Answer**

In [10]:
tf = sentences.iloc[1].count('global') / len(sentences.iloc[1].split())
tf

0.2

In [11]:
idf = np.log(len(sentences) / sentences.str.contains('global').sum())
idf

0.4054651081081644

In [12]:
tf * idf

0.08109302162163289

**Question:** Is this big or small? Is "global" the **best** summary of the second sentence?

### TF-IDF of all words in all documents

On its own, the TF-IDF of a word in a document doesn't really tell us anything; we must compare it to TF-IDFs of other words in that same document.

In [13]:
sentences

0    I really really want global peace
1          I must enjoy global warming
2         We must solve climate change
dtype: object

In [14]:
unique_words = np.unique(sentences.str.split().sum())
unique_words

array(['I', 'We', 'change', 'climate', 'enjoy', 'global', 'must', 'peace',
       'really', 'solve', 'want', 'warming'], dtype='<U7')

In [15]:
tfidf_dict = {}

for word in unique_words:
    re_pat = fr'\b{word}\b'
    tf = sentences.str.count(re_pat) / sentences.str.split().str.len()
    idf = np.log(len(sentences) / sentences.str.contains(re_pat).sum())
    tfidf_dict[word] = tf * idf
    
tfidf = pd.DataFrame(tfidf_dict).set_index(sentences)

In [16]:
tfidf

,I,We,change,climate,enjoy,global,must,peace,really,solve,want,warming
I really really want global peace,0.067578,0.000000,0.000000,0.000000,0.000000,0.067578,0.000000,0.183102,0.366204,0.000000,0.183102,0.000000
I must enjoy global warming,0.081093,0.000000,0.000000,0.000000,0.219722,0.081093,0.081093,0.000000,0.000000,0.000000,0.000000,0.219722
We must solve climate change,0.000000,0.219722,0.219722,0.219722,0.000000,0.000000,0.081093,0.000000,0.000000,0.219722,0.000000,0.000000


### Interpreting TF-IDFs

In [17]:
tfidf

,I,We,change,climate,enjoy,global,must,peace,really,solve,want,warming
I really really want global peace,0.067578,0.000000,0.000000,0.000000,0.000000,0.067578,0.000000,0.183102,0.366204,0.000000,0.183102,0.000000
I must enjoy global warming,0.081093,0.000000,0.000000,0.000000,0.219722,0.081093,0.081093,0.000000,0.000000,0.000000,0.000000,0.219722
We must solve climate change,0.000000,0.219722,0.219722,0.219722,0.000000,0.000000,0.081093,0.000000,0.000000,0.219722,0.000000,0.000000


The above DataFrame tells us that:
- the TF-IDF of `'peace'` in the first sentence is 0.183102,
- the TF-IDF of `'climate'` in the second sentence is 0.

Note that there are two ways that $\text{tfidf}(t, d) = \text{tf}(t, d) \cdot \text{idf}(t)$ can be 0:
- If $t$ appears in every document, because then $\text{idf}(t) = \log (\frac{\text{# documents}}{\text{# documents}}) = \log(1) = 0$.
- If $t$ does not appear in document $d$, because then $\text{tf}(t, d) = \frac{0}{\text{len}(d)} = 0$.

The word that **best summarizes** a document is the word with the highest TF-IDF for that document:

In [18]:
tfidf

,I,We,change,climate,enjoy,global,must,peace,really,solve,want,warming
I really really want global peace,0.067578,0.000000,0.000000,0.000000,0.000000,0.067578,0.000000,0.183102,0.366204,0.000000,0.183102,0.000000
I must enjoy global warming,0.081093,0.000000,0.000000,0.000000,0.219722,0.081093,0.081093,0.000000,0.000000,0.000000,0.000000,0.219722
We must solve climate change,0.000000,0.219722,0.219722,0.219722,0.000000,0.000000,0.081093,0.000000,0.000000,0.219722,0.000000,0.000000


In [19]:
tfidf.idxmax(axis=1)

I really really want global peace    really
I must enjoy global warming           enjoy
We must solve climate change             We
dtype: object

Look closely at the rows of `tfidf` – in documents 1 and 2, the max TF-IDF is not unique!

## Example: State of the Union addresses 🎤

### State of the Union addresses

The 2023 State of the Union address was on February 7th, 2023.
It is an annual message delivered by the president of the United States to a joint session of the United States Congress near the beginning of most calendar years on the current condition of the nation.

### The data

In [21]:
sotu = open('stateoftheunion1790-2023.txt').read()

UnicodeDecodeError: 'gbk' codec can't decode byte 0x94 in position 10113876: illegal multibyte sequence

In [ ]:
len(sotu)

The entire **corpus** (another word for "set of documents") is over 10 million characters long... let's not display it in our notebook.

In [ ]:
print(sotu[:16000])

Each speech is separated by `'***'`.

In [ ]:
speeches = sotu.split('\n***\n')[1:]

In [ ]:
len(speeches)

Note that each "speech" currently contains other information, like the name of the president and the date of the address.

In [ ]:
print(speeches[-1][:1000])

Let's extract just the speech text.

In [ ]:
def extract_struct(speech):
    L = speech.strip().split('\n', maxsplit=3)
    L[3] = re.sub(r"[^A-Za-z' ]", ' ', L[3]).lower()
    return dict(zip(['speech', 'president', 'date', 'contents'], L))

In [ ]:
speeches_df = pd.DataFrame(list(map(extract_struct, speeches)))
speeches_df

### Finding the most important words in each speech

Here, a "document" is a speech. We have 233 documents.

In [ ]:
speeches_df.head()

A rough sketch of what we'll compute:

```
for each word t:
    for each speech d:
        compute tfidf(t, d)
```

In [ ]:
unique_words = pd.Series(speeches_df['contents'].str.split().sum()).value_counts()
unique_words

In [ ]:
unique_words = unique_words.iloc[:500].index

tfidf_dict = {}
tf_denom = speeches_df['contents'].str.split().str.len()
for word in unique_words:
    re_pat = fr' {word} ' # Imperfect pattern for speed.
    tf = speeches_df['contents'].str.count(re_pat) / tf_denom
    idf = np.log(len(speeches_df) / speeches_df['contents'].str.contains(re_pat).sum())
    tfidf_dict[word] =  tf * idf

In [ ]:
tfidf = pd.DataFrame(tfidf_dict)
tfidf.head()

Note that the TF-IDFs of many common words are all 0!

### Summarizing speeches

By using `idxmax`, we can find the word with the highest TF-IDF in each speech.

In [ ]:
summaries = tfidf.idxmax(axis=1)
summaries

What if we want to see the 5 words with the highest TF-IDFs, for each speech?

In [ ]:
def five_largest(row):
    return list(row.index[row.argsort()][-5:])

In [ ]:
keywords = tfidf.apply(five_largest, axis=1)
keywords_df = pd.concat([
    speeches_df['president'],
    speeches_df['date'],
    keywords
], axis=1)

Run the cell below to see every single row of `keywords_df`.

In [ ]:
with pd.option_context('display.max_rows', 300):
    display(keywords_df)

### Aside: What if we remove the $\log$ from $\text{idf}(t)$?

Let's try it and see what happens.

In [ ]:
tfidf_nl_dict = {}
tf_denom = speeches_df['contents'].str.split().str.len()
for word in unique_words:
    re_pat = fr' {word} ' # Imperfect pattern for speed.
    tf = speeches_df['contents'].str.count(re_pat) / tf_denom
    idf_nl = len(speeches_df) / speeches_df['contents'].str.contains(re_pat).sum()
    tfidf_nl_dict[word] =  tf * idf_nl

In [ ]:
tfidf_nl = pd.DataFrame(tfidf_nl_dict)
tfidf_nl.head()

In [ ]:
keywords_nl = tfidf_nl.apply(five_largest, axis=1)
keywords_nl_df = pd.concat([
    speeches_df['president'],
    speeches_df['date'],
    keywords_nl
], axis=1)
keywords_nl_df